In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import os, sys
import pandas as pd
from dask import bag as db
from dask_k8 import DaskCluster
from dask.distributed import Client

In [4]:
sys.path.append('../')

In [5]:
import sanity_check

In [6]:
from sanity_check.contents.s3_data import S3_CANONICAL_DATA_BUCKET
from sanity_check.contents.checks import run_checks_canonical

In [7]:
from sanity_check.contents.s3_data import list_files_rebuilt, fetch_issue_ids_rebuilt
from sanity_check.contents.s3_data import S3_CANONICAL_DATA_BUCKET, fetch_issue_ids
from sanity_check.contents.s3_data import S3_REBUILT_DATA_BUCKET, fetch_issues
# from sanity_check.contents.sync import check_sync_db, configure_db_ingestion
# from sanity_check.contents.sync import check_sync_rebuilt
from sanity_check.contents.checks import run_checks_canonical

In [8]:
from impresso_commons.utils.kube import (make_scheduler_configuration,
                                         make_worker_configuration)

## Create dask k8 cluster

In [9]:
docker_image_uri = "ic-registry.epfl.ch/dhlab/impresso_data-sanity-check:v1"

In [10]:
worker_config = """
containers:
    - image: ic-registry.epfl.ch/dhlab/impresso_data-sanity-check:v1
      args: [dask-worker, $(DASK_SCHEDULER_ADDRESS), --nthreads, '1', --no-bokeh, --memory-limit, 10G, --death-timeout, '120']
      imagePullPolicy: Always
      name: dask-worker
      env:
        - name: POD_IP
          valueFrom:
            fieldRef:
              fieldPath: status.podIP
        - name: POD_NAME
          valueFrom:
            fieldRef:
              fieldPath: metadata.name
        - name: EXTRA_PIP_PACKAGES
          value: 
        - name: EXTRA_CONDA_PACKAGES
          value:
        - name: SE_ACCESS_KEY
          value: 18c8f162ee474359ba3a6dbf08edca0f
        - name: SE_SECRET_KEY
          value: 733f24dbbef045b480da0fdd088a92b6
      resources:
        requests:
          cpu: 1
          memory: 10G
        limits:
          cpu: 1
          memory: 10G
"""

In [16]:
worker_config = make_worker_configuration(
    docker_image=docker_image_uri,
    memory="100G"
)

In [17]:
cluster = DaskCluster(
    namespace="dhlab",
    cluster_id="impresso-sanitycheck",
    scheduler_pod_spec=make_scheduler_configuration(),
    worker_pod_spec=worker_config
)

In [18]:
cluster.create()
cluster.scale(10, blocking=False)

Scheduler: tcp://10.90.50.9:18637
Dashboard: http://10.90.50.9:8598


In [22]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.utils - 

distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.client - ERROR - Failed to reconnect to sc

distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 998, in _reconnect
    await self._close()
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 1268,

In [19]:
dask_client = cluster.make_dask_client()

In [20]:
# dask_client = Client()

In [21]:
dask_client

Client Scheduler: tcp://10.90.50.9:18637 Dashboard: http://10.90.50.9:8787/status,Cluster Workers: 10 Cores: 10 Memory: 1000.00 GB


In [35]:
dask_client.get_versions(check=True)

ValueError: unknown locale: UTF-8

## Run checks

In [33]:
from sanity_check.contents.checks import check_inconsistent_page_ids
from sanity_check.contents.checks import check_duplicated_issues_IDs
from sanity_check.contents.checks import check_duplicated_content_item_IDs
from sanity_check.contents.s3_data import fetch_issues, list_newspapers

In [16]:
canonical_bucket_name = "s3://original-canonical-testing"

In [19]:
canonical_issues_bag = fetch_issues(
    canonical_bucket_name,
    compute=False
).filter(
    lambda i: len(i) > 0
)

Fetching list of newspapers from s3://original-canonical-testing
original-canonical-testing contains 66 newspapers
s3://original-canonical-testing contains 2101 .bz2 files with issues
Fetching issue ids from 2101 .bz2 files (compute=False)


In [20]:
canonical_issues_bag.count().compute()

284898

In [21]:
# 3) verify the consistency of page IDs
page_inconsistencies_df = check_inconsistent_page_ids(canonical_bucket_name)

Fetching list of newspapers from s3://original-canonical-testing
original-canonical-testing contains 66 newspapers
s3://original-canonical-testing contains 2101 .bz2 files with issues
Fetching issue ids from 2101 .bz2 files (compute=False)
Fetching page IDs from issues
Fetching list of newspapers from s3://original-canonical-testing
original-canonical-testing contains 66 newspapers
s3://original-canonical-testing contains 2101 .bz2 files with issues
Fetching issue ids from 2101 .bz2 files (compute=False)
Fetching list of newspapers from s3://original-canonical-testing
original-canonical-testing contains 66 newspapers
s3://original-canonical-testing contains 283860 .bz2 files with pages


In [22]:
print(page_inconsistencies_df.shape[0])
print(page_inconsistencies_df.newspaper_id.unique())

170
['excelsior' 'lafronde' 'luxwort' 'oeuvre']


In [30]:
page_inconsistencies_df[page_inconsistencies_df.newspaper_id=='luxwort']

,from_issues,from_pages,newspaper_id
id,,,
luxwort-1860-10-07-a-p0003,True,NaN,luxwort
luxwort-1860-11-01-a-p0002,True,NaN,luxwort


In [27]:
# 1) verify that there are not duplicated issue IDs
duplicated_issues_df = check_duplicated_issues_IDs(canonical_issues_bag)
duplicated_issues_df.newspaper_id.unique()

1032 duplicated IDs were found


array(['JDG', 'excelsior', 'lafronde', 'GDL', 'oeuvre'], dtype=object)

In [26]:
duplicated_issues_df.newspaper_id.unique()

array(['JDG', 'excelsior', 'lafronde', 'GDL', 'oeuvre'], dtype=object)

In [24]:
# 2) verify that there are not duplicated content item IDs
duplicated_ci_df = check_duplicated_content_item_IDs(canonical_issues_bag)

Found 149681 duplicated content item IDs, belonging to 5 journals(JDG, excelsior, lafronde, GDL, oeuvre)


In [28]:
duplicated_ci_df.newspaper_id.unique()

array(['JDG', 'excelsior', 'lafronde', 'GDL', 'oeuvre'], dtype=object)

In [3]:
# now print the list of newspapers with problems, and those without

In [34]:
newspapers = list_newspapers(canonical_bucket_name)

Fetching list of newspapers from s3://original-canonical-testing
original-canonical-testing contains 66 newspapers


In [55]:
newspapers_with_issues = set(duplicated_ci_df.newspaper_id.unique()).union(
    page_inconsistencies_df.newspaper_id.unique()
).union(
    duplicated_ci_df.newspaper_id.unique()
).union(
    duplicated_issues_df.newspaper_id.unique()
)

In [57]:
greenlighted_newspapers = set(newspapers).difference(nps_with_issues)

In [59]:
len(greenlighted_newspapers)

60

In [63]:
with open('../../impresso-processing/data/greenlight-staging-20200424.txt', 'w') as f:
    f.write("\n".join(greenlighted_newspapers))

## Check for long articles

In [16]:
from impresso_commons.utils.s3 import IMPRESSO_STORAGEOPT, fixed_s3fs_glob
from dask import bag as db
import json

In [163]:
rebuilt_bucket = "s3://canonical-rebuilt-release"
newspaper = "indeplux"

In [164]:
rebuilt_files = fixed_s3fs_glob(f'{rebuilt_bucket}/{newspaper}/*.bz2')

In [165]:
len(rebuilt_files)

61

In [166]:
from collections import Counter

def find_ngrams(text, ngram_size, threshold):
    
    # create ngrams of a given size
    sentence = text.split()
    N = ngram_size
    grams = [
        "/".join(sentence[i:i+N])
        for i in range(len(sentence)-N+1)
    ]
    
    # check for long repeated chunks of text
    filtered_ngrams = []
    for gram, count in Counter(grams).items():
        if count >= threshold:
            filtered_ngrams.append((gram, count))
    
    return filtered_ngrams

In [167]:
def check_repeated_text(content_item):

    if "ft" not in content_item:
        return None

    text = content_item['ft']
    repeated_long_grams = find_ngrams(text, 100, 2)

    if repeated_long_grams:
        return {
            "ci_id": content_item['id'],
            "n_repeated_grams": len(repeated_long_grams)
        }
    else:
        return None

In [ ]:
from dask import bag as db
from impresso_commons.utils.s3 import IMPRESSO_STORAGEOPT

db.read_text(
    "s3://canonical-rebuilt-release/JDG/JDG-1998.jsonl.bz2",
    storage_options=IMPRESSO_STORAGEOPT
).map(
    json.loads
).take(1)

In [168]:
long_content_items = db.read_text(
    rebuilt_files,
    storage_options=IMPRESSO_STORAGEOPT
).map(
    json.loads
).filter(
    lambda i: i is not None and "ft" in i
).map(
    lambda i: {"id":i["id"], "ft":i["ft"]}
).map(
    check_repeated_text
).filter(
    lambda x: x is not None
).compute()

In [31]:
dask_client.cancel(content_items)

In [28]:
dask_client.get_futures_error(content_items)

(<function distributed.worker.execute_task(task)>,
 ((<function dask.bag.core.reify(seq)>,
   (<function dask.bag.core.map_chunk(f, iters, iter_kwarg_keys=None, kwargs=None)>,
    <function __main__.<lambda>(i)>,
    [(filter,
      <function __main__.<lambda>(i)>,
      (<function dask.bag.core.map_chunk(f, iters, iter_kwarg_keys=None, kwargs=None)>,
       <function json.loads(s, *, encoding=None, cls=None, object_hook=None, parse_float=None, parse_int=None, parse_constant=None, object_pairs_hook=None, **kw)>,
       [(<function _operator.getitem>,
         "('split-3e91f7f76ee7a3e6d778ccce528ada99', 14)",
         0)],
       None,
       {}))],
    None,
    {})),),
 {},
 ("('split-3e91f7f76ee7a3e6d778ccce528ada99', 14)",))

In [169]:
long_content_items = content_items.map(
    check_repeated_text
).filter(lambda x: x is not None).compute()

In [51]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/Users/matteo/.pyenv/versions/3.6.8/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/Users/matteo/.pyenv/versions/3.6.8/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/Users/matteo/.pyenv/versions/3.6.8/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/Users/matteo/.pyenv/versions/3.6.8/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 998, in _reconnect
    await self._close()
  File "/Users/matteo/.pyenv/versions/3.

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/Users/matteo/.pyenv/versions/3.6.8/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/Users/matteo/.pyenv/versions/3.6.8/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/Users/matteo/.pyenv/versions/3.6.8/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/Users/matteo/.pyenv/versions/3.6.8/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 998, in _reconnect
    await self._close()
  File "/Users/matteo/.pyenv/versions/3.

distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/Users/matteo/.pyenv/versions/3.6.8/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/Users/matteo/.pyenv/versions/3.6.8/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/Users/matteo/.pyenv/versions/3.6.8/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/Users/matteo/.pyenv/versions/3.6.8/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 998, in _reconnect
    await self._close()
  File "/Users/matteo/.pyenv/versions/3.6.8/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 1268, in _

In [170]:
len(long_content_items)

29

In [171]:
long_content_items

[{'ci_id': 'indeplux-1872-04-06-a-i0005', 'n_repeated_grams': 95},
 {'ci_id': 'indeplux-1874-01-24-a-i0001', 'n_repeated_grams': 341},
 {'ci_id': 'indeplux-1876-10-28-a-i0001', 'n_repeated_grams': 28},
 {'ci_id': 'indeplux-1876-10-28-a-i0011', 'n_repeated_grams': 172},
 {'ci_id': 'indeplux-1876-10-29-a-i0001', 'n_repeated_grams': 102},
 {'ci_id': 'indeplux-1879-08-20-a-i0007', 'n_repeated_grams': 193},
 {'ci_id': 'indeplux-1879-11-07-a-i0008', 'n_repeated_grams': 128},
 {'ci_id': 'indeplux-1879-11-12-a-i0002', 'n_repeated_grams': 259},
 {'ci_id': 'indeplux-1880-06-06-a-i0001', 'n_repeated_grams': 178},
 {'ci_id': 'indeplux-1881-09-24-a-i0014', 'n_repeated_grams': 502},
 {'ci_id': 'indeplux-1882-05-12-a-i0008', 'n_repeated_grams': 39},
 {'ci_id': 'indeplux-1882-05-23-a-i0009', 'n_repeated_grams': 24},
 {'ci_id': 'indeplux-1882-07-23-a-i0008', 'n_repeated_grams': 214},
 {'ci_id': 'indeplux-1882-07-21-a-i0015', 'n_repeated_grams': 319},
 {'ci_id': 'indeplux-1884-04-11-a-i0005', 'n_repeate

In [74]:
ci_id = long_content_items[1]['ci_id']

In [172]:
ci_id = "indeplux-1890-05-10-a-i0015"

In [151]:
from impresso_commons.utils.s3 import alternative_read_text

In [152]:
def get_issue(issue_id, bucket_name):
    newspaper, year = issue_id.split('-')[:2]
    file = f'{bucket_name}/{newspaper}/issues/{newspaper}-{year}-issues.jsonl.bz2'
    print(file)
    return db.from_sequence(alternative_read_text(
        file,
        IMPRESSO_STORAGEOPT
    )).map(
        json.loads
    ).filter(
        lambda x: x['id'] == issue_id
    ).compute()

In [153]:
def get_page(page_id, bucket_name):
    newspaper, year = page_id.split('-')[:2]
    issue = "-".join(page_id.split('-')[:-1])
    file = f'{bucket_name}/{newspaper}/pages/{newspaper}-{year}/{issue}-pages.jsonl.bz2'
    print(file)
    data = db.from_sequence(alternative_read_text(
        file,
        IMPRESSO_STORAGEOPT
    )).map(
        json.loads
    ).filter(
        lambda x: x['id'] == page_id
    ).compute()
    
    if len(data) > 0:
        return data[0]
    else:
        return None

In [154]:
def get_content_item(ci_id, bucket_name):
    newspaper, year = ci_id.split('-')[:2]
    file = f'{bucket_name}/{newspaper}/{newspaper}-{year}.jsonl.bz2'
    
    data = db.from_sequence(alternative_read_text(
        file,
        IMPRESSO_STORAGEOPT
    )).map(
        json.loads
    ).filter(
        lambda x: x['id'] == ci_id
    ).compute()
    
    if len(data) > 0:
        return data[0]
    else:
        return None

In [173]:
tmp = get_content_item(ci_id, "s3://canonical-rebuilt-release")

reading s3://canonical-rebuilt-release/indeplux/indeplux-1890.jsonl.bz2


In [186]:
tmp['id']

'indeplux-1890-05-10-a-i0015'

In [187]:
tmp['pp']

[3]

In [182]:
page = get_page("indeplux-1890-05-10-a-p0003", "s3://original-canonical-staging")

s3://original-canonical-staging/indeplux/pages/indeplux-1890/indeplux-1890-05-10-a-pages.jsonl.bz2
reading s3://original-canonical-staging/indeplux/pages/indeplux-1890/indeplux-1890-05-10-a-pages.jsonl.bz2


In [183]:
page

{'id': 'indeplux-1890-05-10-a-p0003',
 'cdt': '2019-10-16 11:31:46',
 'r': [{'c': [192, 266, 755, 174],
   'p': [{'c': [192, 266, 755, 174],
     'l': [{'c': [198, 270, 745, 43],
       't': [{'c': [198, 277, 60, 31], 'tx': 'que'},
        {'c': [283, 270, 43, 31], 'tx': 'les'},
        {'c': [351, 271, 140, 34], 'tx': 'ouvriers'},
        {'c': [517, 275, 55, 29], 'tx': 'ont'},
        {'c': [596, 270, 186, 43], 'tx': 'consignées'},
        {'c': [805, 270, 77, 31], 'tx': 'dans'},
        {'c': [902, 281, 42, 22], 'tx': 'un'}]},
      {'c': [199, 314, 745, 43],
       't': [{'c': [199, 314, 244, 37], 'tx': 'procès-verbal.'},
        {'c': [493, 317, 49, 31], 'tx': 'On'},
        {'c': [579, 316, 113, 42], 'tx': 'espère'},
        {'c': [728, 327, 61, 28], 'tx': 'que'},
        {'c': [826, 315, 42, 34], 'tx': 'M.'},
        {'c': [904, 315, 40, 31], 'tx': 'de'}]},
      {'c': [198, 356, 748, 47],
       't': [{'c': [198, 356, 137, 35], 'tx': 'Wendel'},
        {'c': [360, 362, 188, 41]

In [184]:
issue = get_issue("indeplux-1890-05-10-a", "s3://original-canonical-staging")

s3://original-canonical-staging/indeplux/issues/indeplux-1890-issues.jsonl.bz2
reading s3://original-canonical-staging/indeplux/issues/indeplux-1890-issues.jsonl.bz2


In [185]:
issue

[{'cdt': '2019-10-16 11:31:46',
  'i': [{'m': {'id': 'indeplux-1890-05-10-a-i0001',
     'l': 'fr',
     'tp': 'article',
     'pp': [1, 3],
     't': 'Dépêches et Dernières Nouvelles. Agence libre.'},
    'l': {'id': 'MODSMD_ARTICLE1',
     'parts': [{'comp_role': 'heading',
       'comp_id': 'ART2-1',
       'comp_fileid': 'ALTO00001',
       'comp_page_no': 1},
      {'comp_role': 'heading',
       'comp_id': 'ART2-2',
       'comp_fileid': 'ALTO00001',
       'comp_page_no': 1},
      {'comp_role': 'body',
       'comp_id': 'ART2-3',
       'comp_fileid': 'ALTO00001',
       'comp_page_no': 1},
      {'comp_role': 'body',
       'comp_id': 'ART2-4',
       'comp_fileid': 'ALTO00001',
       'comp_page_no': 1},
      {'comp_role': 'body',
       'comp_id': 'ART2-5',
       'comp_fileid': 'ALTO00003',
       'comp_page_no': 3},
      {'comp_role': 'body',
       'comp_id': 'ART2-6',
       'comp_fileid': 'ALTO00003',
       'comp_page_no': 3},
      {'comp_role': 'body',
       'comp

In [177]:
ngrams = find_ngrams(tmp['ft'], 100, 2)

In [188]:
ngrams[:10]

[("un/homme/d’une/habileté/supérieure/et/d’une/audace/extraordinaire,/mais/qui/tenait/un/compte/insuffisant/des/droits/du/prochain;/la/manière/même/dont/s'était/contracté/son/mariage/est/caractéristique/de/ses/procédés/de/gouvernement.»/Au/cours/d’un/voyage/électoral,/alors/qu’il/était/candidat/à/la/présidence,/il/avait/eu/l’occasion/de/voir/dans/un/couvent/une/jeune/fille/parfaitement/belle./Il/s’enquit/de/son/nom,/de/sa/famille/et/écrivit/au/père/que/l’enfant/lui/plaisait/beaucoup/et/qu’il/désirait/l’épouser/quand/son/éducation/sera/terminée./«La/famille/avait/des/prétentions/aristocratiques/;/elle/dédaigna/les/ouvertures/de/Barrios/et",
  2),
 ("homme/d’une/habileté/supérieure/et/d’une/audace/extraordinaire,/mais/qui/tenait/un/compte/insuffisant/des/droits/du/prochain;/la/manière/même/dont/s'était/contracté/son/mariage/est/caractéristique/de/ses/procédés/de/gouvernement.»/Au/cours/d’un/voyage/électoral,/alors/qu’il/était/candidat/à/la/présidence,/il/avait/eu/l’occasion/de/voir/dans/

In [178]:
len(ngrams)

300

## Check canonical (page IDs)

In [23]:
from sanity_check.contents.checks import check_inconsistent_page_ids

In [24]:
inconsistencies_df = check_inconsistent_page_ids("s3://original-canonical-testing")

Fetching list of newspapers from s3://original-canonical-testing
original-canonical-testing contains 66 newspapers
s3://original-canonical-testing contains 2101 .bz2 files with issues
Fetching issue ids from 2101 .bz2 files (compute=False)
Fetching page IDs from issues
Fetching list of newspapers from s3://original-canonical-testing
original-canonical-testing contains 66 newspapers
s3://original-canonical-testing contains 2101 .bz2 files with issues
Fetching issue ids from 2101 .bz2 files (compute=False)
Fetching list of newspapers from s3://original-canonical-testing
original-canonical-testing contains 66 newspapers
s3://original-canonical-testing contains 283860 .bz2 files with pages


TypeError: from_sequence() got an unexpected keyword argument 'n_partitions'

### Pages from issue JSON

In [57]:
canonical_bucket = "s3://original-canonical-staging"

In [64]:
from collections import Counter

In [65]:
issue_bag = fetch_issues(
    canonical_bucket,
    compute=False
)

Fetching list of newspapers from s3://original-canonical-staging
original-canonical-staging contains 78 newspapers
s3://original-canonical-staging contains 3101 .bz2 files
Fetching issue ids from 3101 .bz2 files (compute=False)


In [66]:
page_ids_from_issues = issue_bag.map(lambda i: i['pp']).flatten().compute()

In [67]:
len(page_ids_from_issues)

5526792

In [68]:
len(set(page_ids_from_issues))

5510159

In [69]:
duplicates = []
for page_id, count in Counter(page_ids_from_issues).items():
    if count > 1:
        duplicates.append(page_id)

In [70]:
len(duplicates)

16633

In [71]:
duplicates

['JDG-1973-07-07-a-p0001',
 'JDG-1973-07-07-a-p0002',
 'JDG-1973-07-07-a-p0003',
 'JDG-1973-07-07-a-p0004',
 'JDG-1973-07-07-a-p0005',
 'JDG-1973-07-07-a-p0006',
 'JDG-1973-07-07-a-p0007',
 'JDG-1973-07-07-a-p0008',
 'JDG-1973-07-07-a-p0009',
 'JDG-1973-07-07-a-p0010',
 'JDG-1973-07-07-a-p0011',
 'JDG-1973-07-07-a-p0012',
 'JDG-1973-07-07-a-p0013',
 'JDG-1973-07-07-a-p0014',
 'JDG-1973-07-07-a-p0015',
 'JDG-1973-07-07-a-p0016',
 'JDG-1973-07-07-a-p0017',
 'JDG-1973-07-07-a-p0018',
 'JDG-1973-07-07-a-p0019',
 'JDG-1973-07-07-a-p0020',
 'JDG-1973-07-07-a-p0021',
 'JDG-1973-07-07-a-p0022',
 'JDG-1973-07-07-a-p0023',
 'JDG-1973-07-07-a-p0024',
 'JDG-1973-07-20-a-p0001',
 'JDG-1973-07-20-a-p0002',
 'JDG-1973-07-20-a-p0003',
 'JDG-1973-07-20-a-p0004',
 'JDG-1973-07-20-a-p0005',
 'JDG-1973-07-20-a-p0006',
 'JDG-1973-07-20-a-p0007',
 'JDG-1973-07-20-a-p0008',
 'JDG-1973-07-20-a-p0009',
 'JDG-1973-07-20-a-p0010',
 'JDG-1973-07-20-a-p0011',
 'JDG-1973-07-20-a-p0012',
 'JDG-1973-07-20-a-p0013',
 

### Pages from pages JSON

In [18]:
dask_client

Client Scheduler: tcp://10.90.50.17:8006 Dashboard: http://10.90.50.17:8787/status,Cluster Workers: 100 Cores: 100 Memory: 1000.00 GB


In [22]:
import json
from sanity_check.contents.s3_data import list_pages, list_newspapers, fetch_page_ids
from impresso_commons.utils.s3 import IMPRESSO_STORAGEOPT, fixed_s3fs_glob, alternative_read_text
from impresso_commons.utils import chunk

In [20]:
%%time
page_files = list_pages('s3://original-canonical-testing/')

Fetching list of newspapers from s3://original-canonical-testing/
original-canonical-testing contains 66 newspapers
s3://original-canonical-testing/ contains 283860 .bz2 files with pages
CPU times: user 3min 54s, sys: 3.51 s, total: 3min 58s
Wall time: 8min


In [21]:
page_ids = db.from_sequence(page_files, partition_size=100).map(
    alternative_read_text, IMPRESSO_STORAGEOPT
).flatten().map(json.loads).filter(
    lambda i: len(i) > 0
).pluck('id')

In [22]:
df_page_ids_from_pages = page_ids.map(
        lambda id: {"id": id, "from_pages": True}
    ).to_dataframe().set_index('id').persist()

In [23]:
df_page_ids_from_pages.shape[0].compute()

3438969

In [24]:
page_ids_from_issues = fetch_page_ids(
    's3://original-canonical-testing/',
    source="issues"
)

Fetching list of newspapers from s3://original-canonical-testing/
original-canonical-testing contains 66 newspapers
s3://original-canonical-testing/ contains 2101 .bz2 files with issues
Fetching issue ids from 2101 .bz2 files (compute=False)
Fetching page IDs from issues


In [25]:
df_page_ids_from_issues = (
        db.from_sequence(set(page_ids_from_issues))
        .map(lambda id: {"id": id, "from_issues": True})
        .to_dataframe()
        .set_index('id')
        .repartition(npartitions=100)
        .persist()
    )

In [26]:
df_pages = df_page_ids_from_issues.join(
    df_page_ids_from_pages,
    how='outer'
).compute()

In [27]:
df_pages['newspaper_id'] = df_pages.index.map(lambda z: z.split('-')[0])

In [29]:
df_pages[~(df_pages.from_pages == df_pages.from_issues)].shape

(202, 3)

In [35]:
df_pages

,from_issues,from_pages,newspaper_id
id,,,
0002088-1832-11-23-a-p0001,True,True,0002088
0002088-1832-11-23-a-p0002,True,True,0002088
0002088-1832-11-23-a-p0003,True,True,0002088
0002088-1832-11-23-a-p0004,True,True,0002088
0002088-1832-11-23-a-p0005,True,True,0002088
0002088-1832-11-23-a-p0006,True,True,0002088
0002088-1832-11-23-a-p0007,True,True,0002088
0002088-1832-11-23-a-p0008,True,True,0002088
0002088-1832-11-27-a-p0001,True,True,0002088


In [38]:
inconsistencies_df = df_pages[~(df_pages.from_pages == df_pages.from_issues)]

In [45]:
set(df_pages.newspaper_id.unique()) - set(inconsistencies_df.newspaper_id.unique())

{'0002088',
 '0002244',
 'BLB',
 'BNN',
 'CON',
 'DFS',
 'DTT',
 'DVF',
 'EZR',
 'FCT',
 'FZG',
 'FedGazDe',
 'FedGazFr',
 'GAV',
 'GAZ',
 'GDL',
 'HRV',
 'JDG',
 'LAB',
 'LLE',
 'LLS',
 'MGS',
 'NTS',
 'NZG',
 'OIZ',
 'SAX',
 'SDT',
 'SGZ',
 'SMZ',
 'SRT',
 'VDR',
 'VHT',
 'WHD',
 'ZBT',
 'actionfem',
 'arbeitgeber',
 'armeteufel',
 'avenirgdl',
 'buergerbeamten',
 'courriergdl',
 'deletz1893',
 'demitock',
 'diekwochen',
 'dunioun',
 'gazgrdlux',
 'indeplux',
 'kommmit',
 'landwortbild',
 'lunion',
 'luxembourg1935',
 'luxland',
 'luxzeit1844',
 'luxzeit1858',
 'marieclaire',
 'obermosel',
 'onsjongen',
 'schmiede',
 'tageblatt',
 'volkfreu1869',
 'waechtersauer',
 'waeschfra'}

In [40]:
inconsistencies_df.groupby('newspaper_id').size()

newspaper_id
excelsior     80
handelsztg    32
lafronde       2
luxwort        2
oeuvre        86
dtype: int64

In [41]:
inconsistencies_df[inconsistencies_df.newspaper_id=='handelsztg']

,from_issues,from_pages,newspaper_id
id,,,
handelsztg-1884-01-22-a-p0001,NaN,True,handelsztg
handelsztg-1884-01-22-a-p0003,NaN,True,handelsztg
handelsztg-1884-01-22-a-p0004,NaN,True,handelsztg
handelsztg-1884-01-22-a-p0005,NaN,True,handelsztg
handelsztg-1884-01-22-a-p0006,NaN,True,handelsztg
handelsztg-1884-02-27-a-p0001,NaN,True,handelsztg
handelsztg-1884-02-27-a-p0002,NaN,True,handelsztg
handelsztg-1884-02-27-a-p0003,NaN,True,handelsztg
handelsztg-1884-02-27-a-p0004,NaN,True,handelsztg


In [34]:
cluster.close()

## Check canonical (duplicated issue IDs)

In [128]:
from dask.distributed import Client

In [129]:
dask_client = Client()

In [24]:
dask_client.get_versions(check=True)

ValueError: Mismatched versions found

blosc
+----------------------------+---------+
|                            | version |
+----------------------------+---------+
| client                     | None    |
| scheduler                  | 1.8.1   |
| tcp://10.233.107.101:36389 | 1.8.1   |
| tcp://10.233.107.14:34051  | 1.8.1   |
| tcp://10.233.107.30:39961  | 1.8.1   |
| tcp://10.233.107.59:45321  | 1.8.1   |
| tcp://10.233.107.5:43523   | 1.8.1   |
| tcp://10.233.107.80:35235  | 1.8.1   |
| tcp://10.233.107.90:35169  | 1.8.1   |
| tcp://10.233.112.150:37595 | 1.8.1   |
| tcp://10.233.112.178:44157 | 1.8.1   |
| tcp://10.233.112.190:43291 | 1.8.1   |
| tcp://10.233.112.209:38223 | 1.8.1   |
| tcp://10.233.112.223:37999 | 1.8.1   |
| tcp://10.233.112.250:34299 | 1.8.1   |
| tcp://10.233.112.255:37955 | 1.8.1   |
| tcp://10.233.116.195:42703 | 1.8.1   |
| tcp://10.233.116.200:43395 | 1.8.1   |
| tcp://10.233.116.245:39281 | 1.8.1   |
| tcp://10.233.116.247:40215 | 1.8.1   |
| tcp://10.2

In [38]:
dask_client

Client Scheduler: tcp://10.90.50.16:10609 Dashboard: http://10.90.50.16:8787/status,Cluster Workers: 100 Cores: 100 Memory: 1000.00 GB


In [23]:
from sanity_check.contents.s3_data import fetch_issue_ids, fetch_issue_ids_rebuilt, fetch_page_ids
from sanity_check.contents.checks import check_duplicated_issues_IDs

In [17]:
issue_bag = fetch_issues(
    's3://original-canonical-testing/',
    compute=False
)

Fetching list of newspapers from s3://original-canonical-testing/
original-canonical-testing contains 66 newspapers
s3://original-canonical-testing/ contains 2101 .bz2 files with issues
Fetching issue ids from 2101 .bz2 files (compute=False)


In [52]:
from dask import bag
import pandas as pd
from tabulate import tabulate

def local_check_duplicated_issues_IDs(issue_bag: bag.Bag) -> pd.DataFrame:
    """Check that newspaper issue IDs are unique within the corpus."""

    duplicate_issue_ids = (
        issue_bag.pluck("id")
        .frequencies()
        .filter(lambda i: i[1] > 1)
        .map(
            lambda i: {
                "issue_id": i[0],
                "freq": i[1],
                "newspaper_id": i[0].split("-")[0],
            }
        )
        .compute()
    )
    print(f"{len(duplicate_issue_ids)} duplicated IDs were found")
    return pd.DataFrame(duplicate_issue_ids).set_index("issue_id")

def local_check_duplicated_content_item_IDs(issue_bag: bag.Bag) -> pd.DataFrame:
    duplicates = (
        issue_bag.map(lambda issue_json: [ci["m"]["id"] for ci in issue_json["i"]])
        .flatten()
        .frequencies()
        .filter(lambda i: i[1] > 1)
        .map(
            lambda i: {"ci_id": i[0], "freq": i[1], "newspaper_id": i[0].split("-")[0]}
        )
        .compute()
    )

    if duplicates:
        duplicates_df = pd.DataFrame(duplicates).set_index("ci_id")
    else:
        # there are no duplicates
        duplicates_df = pd.DataFrame(columns=["ci_id", "freq", "newspaper_id"])

    print(
        (
            f"Found {duplicates_df.shape[0]} duplicated "
            "content item IDs, belonging to "
            f"{duplicates_df.newspaper_id.unique().size} journals."
        )
    )
    return duplicates_df

In [33]:
duplicated_issues_df = local_check_duplicated_issues_IDs(issue_bag)

1032 duplicated IDs were found


In [39]:
print(tabulate(pd.DataFrame(duplicated_issues_df.groupby('newspaper_id').size())))

---------  ---
GDL        297
JDG        608
excelsior   38
lafronde    31
oeuvre      58
---------  ---


In [53]:
duplicated_ci_df = local_check_duplicated_content_item_IDs(issue_bag)

Found 149681 duplicated content item IDs, belonging to 5 journals.


In [58]:
print(
        (
            f"Found {duplicated_ci_df.shape[0]} duplicated "
            "content item IDs, belonging to "
            f"{duplicated_ci_df.newspaper_id.unique().size} journals"
            f"({', '.join(list(duplicated_ci_df.newspaper_id.unique()))})"
        )
    )

Found 149681 duplicated content item IDs, belonging to 5 journals(GDL, excelsior, JDG, oeuvre, lafronde)


In [70]:
duplicated_ci_df[~duplicated_ci_df.newspaper_id.isin(['GDL', 'JDG'])].shape

(8042, 2)

In [69]:
for ci_id in list(duplicated_ci_df[~duplicated_ci_df.newspaper_id.isin(['GDL', 'JDG'])].index):
    print(f"- {ci_id}")

- excelsior-1913-10-03-a-i0001
- excelsior-1913-10-03-a-i0002
- excelsior-1913-10-03-a-i0003
- excelsior-1913-10-03-a-i0004
- excelsior-1913-10-03-a-i0005
- excelsior-1913-10-03-a-i0006
- excelsior-1913-10-03-a-i0007
- excelsior-1913-10-03-a-i0008
- excelsior-1913-10-03-a-i0009
- excelsior-1913-10-03-a-i0010
- excelsior-1913-10-03-a-i0011
- excelsior-1913-10-03-a-i0012
- excelsior-1913-10-03-a-i0014
- excelsior-1913-10-03-a-i0013
- excelsior-1913-10-03-a-i0015
- excelsior-1913-10-03-a-i0016
- excelsior-1913-10-03-a-i0018
- excelsior-1913-10-03-a-i0017
- excelsior-1913-10-03-a-i0019
- excelsior-1913-10-03-a-i0020
- excelsior-1913-10-03-a-i0021
- excelsior-1913-10-03-a-i0022
- excelsior-1913-10-03-a-i0024
- excelsior-1913-10-03-a-i0023
- excelsior-1913-10-03-a-i0025
- excelsior-1913-10-03-a-i0027
- excelsior-1913-10-03-a-i0026
- excelsior-1913-10-03-a-i0028
- excelsior-1913-10-03-a-i0029
- excelsior-1913-10-03-a-i0030
- excelsior-1913-10-03-a-i0031
- excelsior-1913-10-03-a-i0032
- excels

- oeuvre-1935-05-31-a-i0142
- oeuvre-1935-05-31-a-i0144
- oeuvre-1935-05-31-a-i0145
- oeuvre-1935-05-31-a-i0146
- oeuvre-1935-05-31-a-i0147
- oeuvre-1935-05-31-a-i0148
- oeuvre-1935-05-31-a-i0149
- oeuvre-1935-05-31-a-i0151
- oeuvre-1935-05-31-a-i0152
- oeuvre-1935-05-31-a-i0150
- oeuvre-1935-05-31-a-i0153
- oeuvre-1935-05-31-a-i0154
- oeuvre-1935-05-31-a-i0155
- oeuvre-1935-05-31-a-i0156
- oeuvre-1935-05-31-a-i0157
- oeuvre-1935-05-31-a-i0158
- oeuvre-1935-06-05-a-i0002
- oeuvre-1935-06-05-a-i0001
- oeuvre-1935-06-05-a-i0004
- oeuvre-1935-06-05-a-i0003
- oeuvre-1935-06-05-a-i0006
- oeuvre-1935-06-05-a-i0005
- oeuvre-1935-06-05-a-i0007
- oeuvre-1935-06-05-a-i0008
- oeuvre-1935-06-05-a-i0009
- oeuvre-1935-06-05-a-i0010
- oeuvre-1935-06-05-a-i0011
- oeuvre-1935-06-05-a-i0012
- oeuvre-1935-06-05-a-i0013
- oeuvre-1935-06-05-a-i0014
- oeuvre-1935-06-05-a-i0015
- oeuvre-1935-06-05-a-i0016
- oeuvre-1935-06-05-a-i0017
- oeuvre-1935-06-05-a-i0018
- oeuvre-1935-06-05-a-i0019
- oeuvre-1935-06-05-

- oeuvre-1935-06-30-a-i0081
- oeuvre-1935-06-30-a-i0082
- oeuvre-1935-06-30-a-i0083
- oeuvre-1935-06-30-a-i0084
- oeuvre-1935-06-30-a-i0085
- oeuvre-1935-06-30-a-i0086
- oeuvre-1935-06-30-a-i0087
- oeuvre-1935-06-30-a-i0088
- oeuvre-1935-06-30-a-i0090
- oeuvre-1935-06-30-a-i0091
- oeuvre-1935-06-30-a-i0092
- oeuvre-1935-06-30-a-i0093
- oeuvre-1935-06-30-a-i0094
- oeuvre-1935-06-30-a-i0095
- oeuvre-1935-06-30-a-i0089
- oeuvre-1935-06-30-a-i0096
- oeuvre-1935-06-30-a-i0097
- oeuvre-1935-06-30-a-i0098
- oeuvre-1935-06-30-a-i0099
- oeuvre-1935-06-30-a-i0100
- oeuvre-1935-06-30-a-i0101
- oeuvre-1935-06-30-a-i0103
- oeuvre-1935-06-30-a-i0102
- oeuvre-1935-06-30-a-i0104
- oeuvre-1935-06-30-a-i0105
- oeuvre-1935-06-30-a-i0106
- oeuvre-1935-06-30-a-i0107
- oeuvre-1935-06-30-a-i0108
- oeuvre-1935-06-30-a-i0109
- oeuvre-1935-06-30-a-i0110
- oeuvre-1935-06-30-a-i0111
- oeuvre-1935-06-30-a-i0112
- oeuvre-1935-06-30-a-i0113
- oeuvre-1935-06-30-a-i0114
- oeuvre-1935-06-30-a-i0115
- oeuvre-1935-06-30-

- lafronde-1903-01-25-a-i0028
- lafronde-1903-01-25-a-i0029
- lafronde-1903-01-25-a-i0030
- lafronde-1903-01-25-a-i0031
- lafronde-1903-01-25-a-i0032
- lafronde-1903-01-25-a-i0033
- lafronde-1903-01-25-a-i0034
- lafronde-1903-01-25-a-i0035
- lafronde-1903-01-25-a-i0036
- lafronde-1903-01-25-a-i0037
- lafronde-1903-01-25-a-i0038
- lafronde-1903-01-25-a-i0039
- lafronde-1903-01-25-a-i0040
- lafronde-1903-01-25-a-i0041
- lafronde-1903-01-25-a-i0042
- lafronde-1903-01-25-a-i0043
- lafronde-1903-01-25-a-i0044
- lafronde-1903-01-25-a-i0045
- lafronde-1903-01-25-a-i0046
- lafronde-1903-01-25-a-i0047
- lafronde-1903-01-25-a-i0048
- lafronde-1903-01-25-a-i0049
- lafronde-1903-01-25-a-i0050
- lafronde-1903-01-25-a-i0051
- lafronde-1903-01-25-a-i0052
- lafronde-1903-01-25-a-i0053
- lafronde-1903-01-25-a-i0054
- lafronde-1903-01-25-a-i0055
- lafronde-1903-01-25-a-i0056
- lafronde-1903-01-25-a-i0057
- lafronde-1903-01-25-a-i0058
- lafronde-1903-01-25-a-i0059
- lafronde-1903-01-25-a-i0060
- lafronde

## Release resources

In [32]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    y

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 998, in _reconnect
    await self._close()
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versio

distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 998, in _reconnect
    await self._close()
  File "/home/romanell/.pyenv/versions/3.6.7/envs/impresso-sanity-check/lib/python3.6/site-packages/distributed/client.py", line 1268,